https://python.langchain.com/v0.2/docs/how_to/tool_calling/

In [1]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
# class Add(BaseModel):
#     """Add two integers together."""

#     a: int = Field(..., description="First integer")
#     b: int = Field(..., description="Second integer")


# class Multiply(BaseModel):
#     """Multiply two integers together."""

#     a: int = Field(..., description="First integer")
#     b: int = Field(..., description="Second integer")


# tools = [Add, Multiply]

# We can bind tool to chat models as follows:

In [3]:
import os

os.environ['OPENAI_API_KEY'] = '<OPENAI_API_KEY>' 

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
llm_forced_to_multiply = llm.bind_tools(tools, tool_choice="multiply")
llm_forced_to_multiply.invoke("what is 2 + 4")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P0Av31snjmiISHWOzcvR5JX6', 'function': {'arguments': '{"a":2,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 85, 'total_tokens': 94}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-10fb11cb-2d3b-4a50-bfd1-52c26ad8e6a5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 4}, 'id': 'call_P0Av31snjmiISHWOzcvR5JX6'}], usage_metadata={'input_tokens': 85, 'output_tokens': 9, 'total_tokens': 94})

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49? and what is 5+6? what is 8*8? what is 8*8+2?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_eUW6keVdKf4olqSSIw6wVWbJ'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_ErGP82rAJxzFxHR0EQx2N26a'},
 {'name': 'Add',
  'args': {'a': 5, 'b': 6},
  'id': 'call_xflVZ4G7zBD3JGAKMqHrzB0g'},
 {'name': 'Multiply',
  'args': {'a': 8, 'b': 8},
  'id': 'call_ojHYJwuhIm9gMoIsxL6DMPmt'},
 {'name': 'Add',
  'args': {'a': {'a': 8, 'b': 8}, 'b': 2},
  'id': 'call_e1WrEqYyljGpQDCc1nyzpfI5'}]

In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke(query)

[Multiply(a=3, b=12),
 Add(a=11, b=49),
 Add(a=5, b=6),
 Multiply(a=8, b=8),
 Add(a=8, b=8)]

In [8]:
query = "What is 3 * 12? Also, what is 11 + 49? and what is 5+6?"
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'multiply', 'args': '', 'id': 'call_9AgCjbSfvWlEsf8RKRwGbG3l', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'add', 'args': '', 'id': 'call_GSRcEvlL04rP9iHvrNkeuv7O', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[{'name': 'add', 'args': '', 'id': 'call_YpQAhBCQMdgBGgj6zagJffEw', 'index': 2}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 2}]
[{'name': None, 'args': ': 5, ', 'id': None, 'index': 2}]
[{'name': None, 'args': '"b": 6', 'id': None, 'index': 2}]
[{'name': None, 'args': '}', 'id': None, 'index': 2}]
[]


In [ ]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

[]
[]
[{'name': 'Multiply', 'args': {}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}]
[{'name': 'Multiply', 'args': {'a': 3}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}, {'name': 'Add', 'args': {}, 'id': 'call_qFNNW5uAxzaBAR1d9lfKgVDf'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_qFNNW5uAxzaBAR1d9lfKgVDf'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_qFNNW5uAxzaBAR1d9lfKgVDf'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_eXISghKP6r3SwALspW1CIDWi'}, 

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    print(tool_call)
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_E8372u4M0rACLWyP6C3dQNeM'}
{'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_g6uLqKc55StJ4orpQDizOroA'}
{'name': 'add', 'args': {'a': 5, 'b': 6}, 'id': 'call_li3pvgfi0BLurQLmiPfdHqFo'}


[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49? and what is 5+6?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E8372u4M0rACLWyP6C3dQNeM', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_g6uLqKc55StJ4orpQDizOroA', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_li3pvgfi0BLurQLmiPfdHqFo', 'function': {'arguments': '{"a": 5, "b": 6}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 96, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8e77634e-db8a-40f1-b378-c02b04b20943-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_E8372u4M0rACLWyP6C3dQNeM'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_g6uLqKc55StJ4orpQDizOroA'}, {'name': 'add', 'args': {'a': 5,

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_rpg4NFV7HvTjrDsguTVkzI2g'}]

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

model_with_tools = model.bind(
    tools=[
        {
            "type": "function",
            "function": {
                "name": "multiply",
                "description": "Multiply two integers together.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First integer"},
                        "b": {"type": "number", "description": "Second integer"},
                    },
                    "required": ["a", "b"],
                },
            },
        }
    ]
)

model_with_tools.invoke("Whats 119 times 8?").tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_cqvHR8YBHu4HEaAeX3Gd1bx1'}]